In [1]:
# On terminal: conda activate python38

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import ipynb.fs.defs.functions as fct
import pickle
import scipy

import warnings
warnings.filterwarnings("error")

In [13]:
# Load IDs
with open('uniqueIDs.pkl', 'rb') as f:
    uniqueIDs = pickle.load(f)

# Fit mod0 for each participant

In [14]:
# Model settings

# Functions
value_fct = fct.rescorla_wagner_noV0
dec_fct = fct.my_softmax

# Store everything
mod_info = {}
mod_info['name'] = 'model0'
mod_info['value_fct'] = value_fct.__name__
mod_info['dec_fct'] = dec_fct.__name__
mod_info['param_names'] = ['alpha', 'beta']
#print(mod['value_fct'].__name__)

# save
all_users_folder = 'data/all_users/mod0/'
file_name = all_users_folder+'mod_parameters.pkl'
with open(file_name, 'wb') as f:
    pickle.dump(mod_info, f)

In [17]:
run_ = True

# Folder
all_users_folder = 'data/all_users/mod0/'
    
if run_:
    
    # Parameter range for initial guess 
    # ['alpha', 'beta']
    param_lower_bound = [0, 0]
    param_upper_bound = [1, 10]

    # Fit
    all_users = {}
    p_hit_per_trial = pd.DataFrame([])
    ev_per_trial = pd.DataFrame([])
    
    for n_part,ID in enumerate(uniqueIDs): 

        # Get data
        user_folder = 'data/user_' + ID + '/'
        df2_cf = pd.read_pickle(user_folder + 'df2_cf.pkl')
        isHit_all_cues, fbs_all_cues, trialNo_all_cues = fct.extract_hits_fbs(df2_cf)

        # Create a new Model object
        mod = fct.Model(mod_name = mod_info['name'],
                     value_fct = value_fct, 
                     dec_fct = dec_fct, 
                     param_names = mod_info['param_names'])

        # Input data to model
        mod.set_data(ID, fbs_all_cues, isHit_all_cues, trialNo_all_cues)

        # Fit model
        mod.fit(param_lower_bound, param_upper_bound, n_iterations=5)

        # Nested dictionnary user data
        all_users[n_part] = {}
        all_users[n_part]['ID']=mod.ID
        all_users[n_part]['nLL']=mod.nLL
        all_users[n_part]['Ntrials']=mod.Ntrials
        all_users[n_part]['Nparams']=len(mod.param_names)
        
        for i in range(0,len(mod.param_names)):
            all_users[n_part][mod.param_names[i]]=mod.param_values[i]
        
        # Concatenated model predictions: p hit
        tmp = pd.DataFrame(mod.p_hit).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        p_hit_per_trial = pd.concat([tmp, p_hit_per_trial], axis=0)
        
        # Concatenated model predictions: EVs
        tmp = pd.DataFrame(mod.v).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        ev_per_trial = pd.concat([tmp, ev_per_trial], axis=0)
    
    # Save mod LLs and parameter values
    mod_fit = pd.DataFrame(all_users).transpose()
    mod_fit.to_pickle(all_users_folder+'mod_param_fits.pkl')
    
    # Save mod predictions
    p_hit_per_trial = p_hit_per_trial.sort_values(by='ID').reset_index(drop=True)
    p_hit_per_trial.to_pickle(all_users_folder+'mod_p_hit_per_trial.pkl')
    ev_per_trial = ev_per_trial.sort_values(by='ID').reset_index(drop=True)
    ev_per_trial.to_pickle(all_users_folder+'mod_ev_per_trial.pkl')
    

In [18]:
mod.PEs

{'HR': array([ 5.        ,  2.80408727, -8.42741892,  5.27375638, -7.04238537,
         6.05050737, -2.60676987,  0.53807796,  0.30176351,  4.16923424,
         2.33817933, -4.68870822,  3.37049059, -2.10977005, -7.18319587,
        -0.02846162,         nan,         nan, -0.01596177,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan]),
 'LR': array([-1.        ,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,  1.43918255,
         4.80711869,  2.69591606, -2.48808322, -1.39536049, -2.78254252,
         0.43950159, -5.75351984,  2.77332565,         nan, -0.44467057,
        -4.24937902,  7.61687408,         nan]),
 'HP': array([-5.        ,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
        